Data Merging from RAW data form link to 12 features.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#########################################################################################################
# 1. Load data set and change date and tag name 
#########################################################################################################

# Read the file
column_names = ['sequence_name', 'tag_identificator', 'timestamp', 'date', 'x-coordinate', 'y-coordinate', 'z-coordinate', 'activity']
data_act = pd.read_csv('ConfLongDemo_JSI.txt',sep=',',header=None, names=column_names)

# Makes the data easier to understand. Replace name of tag_idintificator
data_act = data_act.replace(to_replace= '010-000-024-033', value='ankle_left')
data_act = data_act.replace(to_replace= '010-000-030-096', value='ankle_right')
data_act = data_act.replace(to_replace= '020-000-033-111', value='chest')
data_act = data_act.replace(to_replace= '020-000-032-221', value='belt')

display(data_act)

def data_merger(data_act, sequence_name):
    '''Takes a dataframe and a sequence name, returns the same dataframe merged and interpolated.'''
    #########################################################################################################
    # 2. Create 4 separate datasets for each tag and rename xyz coordinates
    #########################################################################################################
    
    # Start using only one sequence
    data_act = data_act[data_act['sequence_name'].isin([sequence_name])]

    # Split into 4 datasets, one for each sensor (only 2 at first)
    data_chest = data_act[data_act['tag_identificator'].isin(['chest'])]
    data_belt = data_act[data_act['tag_identificator'].isin(['belt'])]
    data_ar = data_act[data_act['tag_identificator'].isin(['ankle_right'])]
    data_al = data_act[data_act['tag_identificator'].isin(['ankle_left'])]

    # Rename the features so that they can be separated after merging
    data_chest.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'c_date', 'chest_x-coordinate', 'chest_y-coordinate', 'chest_z-coordinate', 'chest_activity']
    data_belt.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'b_date', 'belt_x-coordinate', 'belt_y-coordinate', 'belt_z-coordinate', 'belt_activity']
    data_ar.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'ar_date', 'ar_x-coordinate', 'ar_y-coordinate', 'ar_z-coordinate', 'ar_activity']
    data_al.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'al_date', 'al_x-coordinate', 'al_y-coordinate', 'al_z-coordinate', 'al_activity']



    #########################################################################################################
    # 3. Merge each of the 4 onto the original dataset one anfter another and remove original xyz coordinate
    #########################################################################################################

    # We merge each dataset onto the original using timestamp because it is unique (date is NOT unique)
    # https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4
    data_merged = data_act.merge(data_chest, on='timestamp', how='left')
    data_merged = data_merged.merge(data_belt, on='timestamp', how='left')
    data_merged = data_merged.merge(data_ar, on='timestamp', how='left')
    data_merged = data_merged.merge(data_al, on='timestamp', how='left')


    #########################################################################################################
    # 4. Use interpolation to fill in the gaps in our data
    #########################################################################################################
    
    # https://towardsdatascience.com/every-pandas-function-you-can-should-use-to-manipulate-time-series-711cb0c5c749
 


    list_of_coordinates_axles = ['chest_x-coordinate', 'chest_y-coordinate', 'chest_z-coordinate', 'belt_x-coordinate', 'belt_y-coordinate', 'belt_z-coordinate',
                                'ar_x-coordinate', 'ar_y-coordinate', 'ar_z-coordinate', 'al_x-coordinate', 'al_y-coordinate', 'al_z-coordinate']
    
    # Fills in missing data between data_points using interpolate. The remaing points at the beginning and end of each sequence
    # get filled in with ffill and bfill.
    data_merged['date'] = pd.to_datetime(data_merged['date'], format='%d.%m.%Y %H:%M:%S:%f')
    data_merged.set_index("date", inplace=True)

    for coordinate in list_of_coordinates_axles:
        data_merged[coordinate] = data_merged[coordinate].interpolate(method='time')
        data_merged[coordinate] = data_merged[coordinate].ffill()
        data_merged[coordinate] = data_merged[coordinate].bfill()

    # This removes duplicate columns created during merger
    data_merged = data_merged.drop(columns=['x-coordinate', 'y-coordinate', 'z-coordinate', 
                               'tag_identificator_y', 
                              'c_date',  
                              'chest_activity', 'tag_identificator_x', 'b_date', 

                              'belt_activity', 'sequence_name_y', 'tag_identificator_y', 
                              'ar_date',  
                              'ar_activity', 'sequence_name', 'tag_identificator', 'al_date', 
                              'al_activity'])
    
    # We following two steps are a verbose way of removing one of the seuquence names. 
    data_merged.columns = ['sequence_name', 'timestamp', 'activity', 'chest_x-coordinate', 
                             'chest_y-coordinate', 'chest_z-coordinate', 'sequence_name_x', 'belt_x-coordinate', 
                             'belt_y-coordinate', 'belt_z-coordinate', 'ar_x-coordinate', 'ar_y-coordinate', 
                             'ar_z-coordinate', 'al_x-coordinate', 'al_y-coordinate', 'al_z-coordinate']
    data_merged = data_merged.drop(columns=['sequence_name_x'])


    return data_merged

# The code below serves to create a new data frame. It splits the data frame into sequences, creates a new data frame 
# and then concatinates (adds together) the dataframes.

list_of_datasequences = data_act['sequence_name'].unique()
list_of_dataframes = []

# We convert dataframes into merged dataframes, one sequence at a time (we don't want to interpolate using datapoints
# from other sequences).

for sequence in list_of_datasequences:
    list_of_dataframes.append(data_merger(data_act, sequence))

counter = 0
# This adds them all together into one dataframe
merged_data_act = list_of_dataframes[0]
for sequence in range(len(list_of_datasequences)):
    if counter == 0:
        counter = 1
    else:
        merged_data_act = pd.concat([merged_data_act, list_of_dataframes[sequence]])

merged_data_act['date'] = merged_data_act.index
display(merged_data_act)
merged_data_act.to_csv('merged_data_NEW_test.csv', index=False)

sequence_name tag_identificator           timestamp  \
0                A01        ankle_left  633790226051280329   
1                A01             chest  633790226051820913   
2                A01              belt  633790226052091205   
3                A01        ankle_left  633790226052361498   
4                A01       ankle_right  633790226052631792   
...              ...               ...                 ...   
164855           E05       ankle_right  633790146419554374   
164856           E05        ankle_left  633790146419824669   
164857           E05             chest  633790146420094965   
164858           E05       ankle_right  633790146420635550   
164859           E05        ankle_left  633790146420905847   

                           date  x-coordinate  y-coordinate  z-coordinate  \
0       27.05.2009 14:03:25:127      4.062931      1.892434      0.507425   
1       27.05.2009 14:03:25:183      4.291954      1.781140      1.344495   
2       27.05.2009 14:03:25:210      4.359101      1.826456      0.968821   
3       27.05.2009 14:03:25:237      4.087835      1.879999      0.466983   
4       27.05.2009 14:03:25:263      4.324462      2.072460      0.488065   
...                         ...           ...           ...           ...   
164855  27.05.2009 11:50:41:957      3.209474      2.044571      0.062902   
164856  27.05.2009 11:50:41:983      3.386878      2.004729      0.395161   
164857  27.05.2009 11:50:42:010      3.188895      1.915717      1.353087   
164858  27.05.2009 11:50:42:063      3.150169      1.931164      0.055037   
164859  27.05.2009 11:50:42:090      3.209994      1.939577      0.364777   

       activity  
0       walking  
1       walking  
2       walking  
3       walking  
4       walking  
...         ...  
164855  walking  
164856  walking  
164857  walking  
164858  walking  
164859  walking  

[164860 rows x 8 columns]

sequence_name           timestamp activity  \
date                                                                 
2009-05-27 14:03:25.127           A01  633790226051280329  walking   
2009-05-27 14:03:25.183           A01  633790226051820913  walking   
2009-05-27 14:03:25.210           A01  633790226052091205  walking   
2009-05-27 14:03:25.237           A01  633790226052361498  walking   
2009-05-27 14:03:25.263           A01  633790226052631792  walking   
...                               ...                 ...      ...   
2009-05-27 11:50:41.957           E05  633790146419554374  walking   
2009-05-27 11:50:41.983           E05  633790146419824669  walking   
2009-05-27 11:50:42.010           E05  633790146420094965  walking   
2009-05-27 11:50:42.063           E05  633790146420635550  walking   
2009-05-27 11:50:42.090           E05  633790146420905847  walking   

                         chest_x-coordinate  chest_y-coordinate  \
date                                                              
2009-05-27 14:03:25.127            4.291954            1.781140   
2009-05-27 14:03:25.183            4.291954            1.781140   
2009-05-27 14:03:25.210            4.310153            1.777747   
2009-05-27 14:03:25.237            4.328352            1.774353   
2009-05-27 14:03:25.263            4.345878            1.771085   
...                                     ...                 ...   
2009-05-27 11:50:41.957            3.142250            1.935279   
2009-05-27 11:50:41.983            3.165132            1.925682   
2009-05-27 11:50:42.010            3.188895            1.915717   
2009-05-27 11:50:42.063            3.188895            1.915717   
2009-05-27 11:50:42.090            3.188895            1.915717   

                         chest_z-coordinate  belt_x-coordinate  \
date                                                             
2009-05-27 14:03:25.127            1.344495           4.359101   
2009-05-27 14:03:25.183            1.344495           4.359101   
2009-05-27 14:03:25.210            1.333267           4.359101   
2009-05-27 14:03:25.237            1.322038           4.362838   
2009-05-27 14:03:25.263            1.311225           4.366436   
...                                     ...                ...   
2009-05-27 11:50:41.957            1.421157           3.359577   
2009-05-27 11:50:41.983            1.387764           3.359577   
2009-05-27 11:50:42.010            1.353087           3.359577   
2009-05-27 11:50:42.063            1.353087           3.359577   
2009-05-27 11:50:42.090            1.353087           3.359577   

                         belt_y-coordinate  belt_z-coordinate  \
date                                                            
2009-05-27 14:03:25.127           1.826456           0.968821   
2009-05-27 14:03:25.183           1.826456           0.968821   
2009-05-27 14:03:25.210           1.826456           0.968821   
2009-05-27 14:03:25.237           1.841975           0.918404   
2009-05-27 14:03:25.263           1.856920           0.869853   
...                                    ...                ...   
2009-05-27 11:50:41.957           2.027247           1.070798   
2009-05-27 11:50:41.983           2.027247           1.070798   
2009-05-27 11:50:42.010           2.027247           1.070798   
2009-05-27 11:50:42.063           2.027247           1.070798   
2009-05-27 11:50:42.090           2.027247           1.070798   

                         ar_x-coordinate  ar_y-coordinate  ar_z-coordinate  \
date                                                                         
2009-05-27 14:03:25.127         4.324462         2.072460         0.488065   
2009-05-27 14:03:25.183         4.324462         2.072460         0.488065   
2009-05-27 14:03:25.210         4.324462         2.072460         0.488065   
2009-05-27 14:03:25.237         4.324462         2.072460         0.488065   
2009-05-27 14:03:25.263         4.324462         2.072460         0.

Data merging from 4 outlier_free files to 12 features.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#########################################################################################################
# 1. Load data set and change date and tag name 
#########################################################################################################

# Read the file
column_names = ['sequence_name', 'tag_identificator', 'timestamp', 'date', 'x-coordinate', 'y-coordinate', 'z-coordinate', 'activity', 'outlier_prediction']

data_chest = pd.read_csv('Chest edited.txt',sep=',',header=0, names=column_names)
data_belt = pd.read_csv('Belt edited.txt',sep=',',header=0, names=column_names)
data_ar = pd.read_csv('Ankle_right edited.txt',sep=',',header=0, names=column_names)
data_al = pd.read_csv('Ankle_left edited.txt',sep=',',header=0, names=column_names)
#data_act['date'] = pd.to_datetime(data_act['date'], format='%d.%m.%Y %H:%M:%S:%f')


data_act = pd.concat([data_chest, data_belt, data_ar, data_al])
data_act = data_act.drop(columns=['outlier_prediction'])
data_act = data_act.sort_values(by=['sequence_name', 'date'])
display(data_act)

def data_merger(data_act, sequence_name):
    '''Takes a dataframe and a sequence name, returns the same dataframe merged and interpolated.'''
    #########################################################################################################
    # 2. Create 4 separate datasets for each tag and rename xyz coordinates
    #########################################################################################################
    
    # Start using only one sequence
    data_act = data_act[data_act['sequence_name'].isin([sequence_name])]

    # Split into 4 datasets, one for each sensor (only 2 at first)
    # Note that the outlier free files use CAPS inte the tag_identificator...
    data_chest = data_act[data_act['tag_identificator'].isin(['Chest'])]
    data_belt = data_act[data_act['tag_identificator'].isin(['Belt'])]
    data_ar = data_act[data_act['tag_identificator'].isin(['Ankle_right'])]
    data_al = data_act[data_act['tag_identificator'].isin(['Ankle_left'])]

    # Rename the features so that they can be separated after merging
    data_chest.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'c_date', 'chest_x-coordinate', 'chest_y-coordinate', 'chest_z-coordinate', 'chest_activity']
    data_belt.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'b_date', 'belt_x-coordinate', 'belt_y-coordinate', 'belt_z-coordinate', 'belt_activity']
    data_ar.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'ar_date', 'ar_x-coordinate', 'ar_y-coordinate', 'ar_z-coordinate', 'ar_activity']
    data_al.columns = ['sequence_name', 'tag_identificator', 'timestamp', 'al_date', 'al_x-coordinate', 'al_y-coordinate', 'al_z-coordinate', 'al_activity']



    #########################################################################################################
    # 3. Merge each of the 4 onto the original dataset one anfter another and remove original xyz coordinate
    #########################################################################################################

    # We merge each dataset onto the original using timestamp because it is unique (date is NOT unique)
    # https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4
    data_merged = data_act.merge(data_chest, on='timestamp', how='left')
    data_merged = data_merged.merge(data_belt, on='timestamp', how='left')
    data_merged = data_merged.merge(data_ar, on='timestamp', how='left')
    data_merged = data_merged.merge(data_al, on='timestamp', how='left')


    #########################################################################################################
    # 4. Use interpolation to fill in the gaps in our data
    #########################################################################################################
    
    # https://towardsdatascience.com/every-pandas-function-you-can-should-use-to-manipulate-time-series-711cb0c5c749
 


    list_of_coordinates_axles = ['chest_x-coordinate', 'chest_y-coordinate', 'chest_z-coordinate', 'belt_x-coordinate', 'belt_y-coordinate', 'belt_z-coordinate',
                                'ar_x-coordinate', 'ar_y-coordinate', 'ar_z-coordinate', 'al_x-coordinate', 'al_y-coordinate', 'al_z-coordinate']
    
    # Fills in missing data between data_points using interpolate. The remaing points at the beginning and end of each sequence
    # get filled in with ffill and bfill.
    for coordinate in list_of_coordinates_axles:
        data_merged[coordinate] = data_merged[coordinate].interpolate()
        data_merged[coordinate] = data_merged[coordinate].ffill()
        data_merged[coordinate] = data_merged[coordinate].bfill()

    # This removes duplicate columns created during merger
    data_merged = data_merged.drop(columns=['x-coordinate', 'y-coordinate', 'z-coordinate', 
                               'tag_identificator_y', 
                              'c_date',  
                              'chest_activity', 'tag_identificator_x', 'b_date', 

                              'belt_activity', 'sequence_name_y', 'tag_identificator_y', 
                              'ar_date',  
                              'ar_activity', 'sequence_name', 'tag_identificator', 'al_date', 
                              'al_activity'])
    
    # We following two steps are a verbose way of removing one of the seuquence names. 
    data_merged.columns = ['sequence_name', 'timestamp', 'date', 'activity', 'chest_x-coordinate', 
                             'chest_y-coordinate', 'chest_z-coordinate', 'sequence_name_x', 'belt_x-coordinate', 
                             'belt_y-coordinate', 'belt_z-coordinate', 'ar_x-coordinate', 'ar_y-coordinate', 
                             'ar_z-coordinate', 'al_x-coordinate', 'al_y-coordinate', 'al_z-coordinate']
    data_merged = data_merged.drop(columns=['sequence_name_x'])


    return data_merged

# The code below serves to create a new data frame. It splits the data frame into sequences, creates a new data frame 
# and then concatinates (adds together) the dataframes.

list_of_datasequences = data_act['sequence_name'].unique()
list_of_dataframes = []

# We convert dataframes into merged dataframes, one sequence at a time (we don't want to interpolate using datapoints
# from other sequences).

for sequence in list_of_datasequences:
    list_of_dataframes.append(data_merger(data_act, sequence))

counter = 0
# This adds them all together into one dataframe
merged_data_act = list_of_dataframes[0]
for sequence in range(len(list_of_datasequences)):
    if counter == 0:
        counter = 1
    else:
        merged_data_act = pd.concat([merged_data_act, list_of_dataframes[sequence]])

display(merged_data_act)
merged_data_act.to_csv('merged_data_no_outliers.csv', index=False)

sequence_name tag_identificator           timestamp  \
0                A01        Ankle_left  633790226051280329   
1                A01             Chest  633790226051820913   
2                A01              Belt  633790226052091205   
3                A01        Ankle_left  633790226052361498   
4                A01       Ankle_right  633790226052631792   
...              ...               ...                 ...   
164855           E05       Ankle_right  633790146419554374   
164856           E05        Ankle_left  633790146419824669   
164857           E05             Chest  633790146420094965   
164858           E05       Ankle_right  633790146420635550   
164859           E05        Ankle_left  633790146420905847   

                           date  x-coordinate  y-coordinate  z-coordinate  \
0       27.05.2009 14:03:25:127      4.062931      1.892434      0.507425   
1       27.05.2009 14:03:25:183      4.291954      1.781140      1.344495   
2       27.05.2009 14:03:25:210      4.359101      1.826456      0.968821   
3       27.05.2009 14:03:25:237      4.087835      1.879999      0.466983   
4       27.05.2009 14:03:25:263      4.324462      2.072460      0.488065   
...                         ...           ...           ...           ...   
164855  27.05.2009 11:50:41:957      3.209474      2.044571      0.062902   
164856  27.05.2009 11:50:41:983      3.386878      2.004729      0.395161   
164857  27.05.2009 11:50:42:010      3.188895      1.915717      1.353087   
164858  27.05.2009 11:50:42:063      3.150169      1.931164      0.055037   
164859  27.05.2009 11:50:42:090      3.209994      1.939577      0.364777   

       activity  
0       walking  
1       walking  
2       walking  
3       walking  
4       walking  
...         ...  
164855  walking  
164856  walking  
164857  walking  
164858  walking  
164859  walking  

[163210 rows x 8 columns]

sequence_name           timestamp                     date activity  \
0              A01  633790226051280329  27.05.2009 14:03:25:127  walking   
1              A01  633790226051820913  27.05.2009 14:03:25:183  walking   
2              A01  633790226052091205  27.05.2009 14:03:25:210  walking   
3              A01  633790226052361498  27.05.2009 14:03:25:237  walking   
4              A01  633790226052631792  27.05.2009 14:03:25:263  walking   
...            ...                 ...                      ...      ...   
8816           E05  633790146419554374  27.05.2009 11:50:41:957  walking   
8817           E05  633790146419824669  27.05.2009 11:50:41:983  walking   
8818           E05  633790146420094965  27.05.2009 11:50:42:010  walking   
8819           E05  633790146420635550  27.05.2009 11:50:42:063  walking   
8820           E05  633790146420905847  27.05.2009 11:50:42:090  walking   

      chest_x-coordinate  chest_y-coordinate  chest_z-coordinate  \
0               4.291954            1.781140            1.344495   
1               4.291954            1.781140            1.344495   
2               4.309984            1.777778            1.333371   
3               4.328015            1.774416            1.322246   
4               4.346046            1.771054            1.311121   
...                  ...                 ...                 ...   
8816            3.140489            1.936017            1.423725   
8817            3.164692            1.925867            1.388406   
8818            3.188895            1.915717            1.353087   
8819            3.188895            1.915717            1.353087   
8820            3.188895            1.915717            1.353087   

      belt_x-coordinate  belt_y-coordinate  belt_z-coordinate  \
0              4.359101           1.826456           0.968821   
1              4.359101           1.826456           0.968821   
2              4.359101           1.826456           0.968821   
3              4.362803           1.841832           0.918870   
4              4.366505           1.857208           0.868920   
...                 ...                ...                ...   
8816           3.359577           2.027247           1.070798   
8817           3.359577           2.027247           1.070798   
8818           3.359577           2.027247           1.070798   
8819           3.359577           2.027247           1.070798   
8820           3.359577           2.027247           1.070798   

      ar_x-coordinate  ar_y-coordinate  ar_z-coordinate  al_x-coordinate  \
0            4.324462         2.072460         0.488065         4.062931   
1            4.324462         2.072460         0.488065         4.071232   
2            4.324462         2.072460         0.488065         4.079534   
3            4.324462         2.072460         0.488065         4.087835   
4            4.324462         2.072460         0.488065         4.055501   
...               ...              ...              ...              ...   
8816         3.209474         2.044571         0.062902         3.351634   
8817         3.189705         2.006769         0.060280         3.386878   
8818         3.169937         1.968966         0.057659         3.327917   
8819         3.150169         1.931164         0.055037         3.268955   
8820         3.150169         1.931164         0.055037         3.209994   

      al_y-coordinate  al_z-coordinate  
0            1.892434         0.507425  
1            1.888289         0.493945  
2            1.884144         0.480464  
3            1.879999         0.466983  
4            1.835890         0.477998  
...               ...              ...  
8816         1.988035         0.393693  
8817         2.004729         0.395161  
8818         1.983011         0.385033  
8819         1.961294         0.374905  
8820         1.939577         0.364777  

[163210 rows x 16 columns]

Takes data frame from PCA analyses and only chooses the top three.